In [280]:
import numpy as np
from tensorflow.python.ops.numpy_ops.np_dtypes import int64
np.set_printoptions(precision=2)  # reduced display precision on numpy arrays


## generating data sets
this code will take 2 params, the number of samples and the number of features and the code will generate the data sets randomly

In [281]:
def generate_dataset(num_samples = 100, n_features = 10):
    """
    Generates a synthetic dataset with a specified number of features and samples.

    Args:
        num_samples (int): Number of data points to generate.
        n_features (int): Number of features in the dataset.

    Returns:
        X_train (ndarray): Feature matrix of shape (num_samples, n_features).
        y_train (ndarray): Target array of shape (num_samples,).
    """
    # Initialize the feature matrix
    X_train = np.zeros((num_samples, n_features),dtype=int64)

    # Populate features dynamically
    for i in range(n_features):
        if i == 0:  # Feature 1: Area in square feet
            X_train[:, i] = np.random.randint(800, 4000, num_samples)
        elif i == 1:  # Feature 2: Bedrooms (correlated with area)
            X_train[:, i] = np.clip((X_train[:, 0] // 1000) + np.random.randint(-1, 2, num_samples), 1, 5)
        elif i == 2:  # Feature 3: Bathrooms (correlated with bedrooms)
            X_train[:, i] = np.clip(X_train[:, 1] + np.random.randint(-1, 2, num_samples), 1, 3)
        elif i == 3:  # Feature 4: Age of property
            X_train[:, i] = np.random.randint(5, 50, num_samples)
        else:  # Additional features: Random values within a defined range
            X_train[:, i] = np.random.randint(1, 100, num_samples)

    # Generate labels with a relationship to the features
    # Using coefficients for up to 4 features, extendable for additional features
    coefficients = np.array([50, 30, 20, -10] + [5] * (n_features - 4))  # Default coefficients
    noise = np.random.normal(0, 500, num_samples)  # Small noise for realism
    y_train = (X_train @ coefficients[:n_features] + noise).astype(int)

    return X_train, y_train

<a name="toc_15456_3"></a>
## Model Prediction With Multiple Variables
The model's prediction with multiple variables is given by the linear model:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
where $\cdot$ is a vector `dot product`

To demonstrate the dot product, we will implement prediction using (1) and (2).

In [282]:
def predict(W,b,X_record):
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    return np.dot(W,X_record) + b

<a name="toc_15456_4"></a>
# Compute Cost With Multiple Variables
The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


In contrast to previous labs, $\mathbf{w}$ and $\mathbf{x}^{(i)}$ are vectors rather than scalars supporting multiple features.

In [283]:
def cost(W,b,X,y):
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    #iterative code
    m = X.shape[0]
    # c = 0.0
    # for i in range(m):
    #     f_wb_i = np.dot(W,X[i]) + b
    #     c += (f_wb_i - y[i])**2
    # c= c/2*m
    
    #numpy code
    
    #it will iterate over the rows of X and dot it by W and add the b and then it will be stored in f_wb[i]and that's for every row of X  
    F_WB = np.dot(X,W) + b
    Squared_Error = (F_WB  - y) ** 2
    c = np.sum(Squared_Error)/(2*m)
    
    return c

<a name="toc_15456_5"></a>
## 5 Gradient Descent With Multiple Variables
Gradient descent for multiple variables:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, n is the number of features, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m is the number of training examples in the data set

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target value


### Dot Product Explanation

The dot product will return an array, where each element contains the dot product of a record (row) with the weights vector.

#### Example
Consider the following table with 4 features and \(m\) records:

| Feature 1 | Feature 2 | Feature 3 | Feature 4 | Target (y) |
|-----------|-----------|-----------|-----------|------------|
| 10        | 20        | 30        | 40        | 203        |
| ..        | ..        | ..        | ..        | ...        |

and the weights of features

| W of Feature 1 | W of Feature 2 | W of Feature 3 | W of Feature 4 |
|----------------|----------------|----------------|----------------|
| 1              | 0.1            | 2              | -22            | 

#### Dot Product Computation:
the dot product will return an array with m elements [x[i] . w[i], ... ] 
or [ [10,20,30,40] . [1,0.1,2,-22] , ... ]

In [284]:

def partial_derivative(W,b,X,y):
    """
    Compute the partial derivatives of the cost function with respect to W and b.

    Args:
      W (ndarray (n,)): Model parameters (weights)
      b (scalar): Model parameter (bias)
      X (ndarray (m, n)): Input data (m records, n features)
      y (ndarray (m,)): Target values

    Returns:
      DJ_DW (ndarray (n,)): Partial derivatives with respect to W
      dj_db (scalar): Partial derivative with respect to b
    """
    number_of_features = n = X.shape[1]
    number_of_records = m = X.shape[0]
    DJ_DW = np.zeros(number_of_features)
    dj_db = 0.0
    errors = np.dot(X,W) + b - y         #the error for every record
    dj_db = np.sum(errors)/m  
    for j in range(number_of_features):
        DJ_DW[j] = (np.sum(errors * X[:,j]))/m
    return DJ_DW, dj_db


### now into gradient descent code

In [285]:
 def gradient_descent(Inint_W,init_b,alpha,times,X_Training,y_training): 
     
    W = Inint_W
    number_of_features = n = W.shape[0]
    b = init_b
        
    for i in range(times):
        flag = True
        DJ_DW,dj_db = partial_derivative(W,b,X_Training,y_training)
        flag &= (abs(dj_db * alpha)<1e-7)
        b = b - alpha*dj_db
        flag &= ((np.sum(abs(DJ_DW * alpha))/number_of_features) < 1e-7)
        W = W - DJ_DW * alpha
        if(flag):
            print(i); break
    return W,b
 
 def gradient_descent_algorithm(X_Training,y_training):
     number_of_features = X_Training.shape[1]
     Init_W = np.full(number_of_features,0)
     init_b = 0.
     alpha = 5.0e-8
     times = 10000
     return gradient_descent(Init_W,init_b,alpha,times,X_Training,y_training)
        

## Running the Code and taking Predication

In [286]:
num_samples = 10000 ; num_features = 4

x_train, y_train = generate_dataset(num_samples,num_features)
# x_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
# y_train = np.array([460, 232, 178])

W,b = gradient_descent_algorithm(x_train,y_train)

print("the Weights are: ",W)
print(f"the bias is: {b:0.2f}")
print(predict(W,b,np.array([1000,2,3,4])))



the Weights are:  [ 5.00e+01  4.91e-02  3.19e-02 -7.79e-01]
the bias is: 0.00
49949.26179563136
